# Computer Vision
Computer Vision is a branch of artificial intelligence (AI) that explores the development of AI systems that can "see" the world, either in real-time through a camera or by analyzing images and video.

## Using the Computer Vision Cognitive Service

Microsoft Azure includes a number of *cognitive services* that encapsulate common AI functions, including some that can help you build computer vision solutions.

The *Computer Vision* cognitive service provides an obvious starting point for our exploration of computer vision in Azure. It uses pre-trained machine learning models to analyze images and extract information about them.

For example, suppose Adventure Works Cycles has set up a number of cameras around the city to track the cycles that have been rented. By using the Computer Vision service, the images taken by the cameras can be analyzed to provide meaningful descriptions of what they depict.

> **Citation**: The images used in this lab are from the [PASCAL Visual Object Classes (VOC) challenge dataset](http://host.robots.ox.ac.uk/pascal/VOC/voc2012/).

Let's start by creating a **Cognitive Services** resource in your Azure subscription:

1. Open the [Azure portal](https://portal.azure.com), signing in with your Microsoft account.
2. Click the **&#65291;Create a resource** button, search for *Cognitive Services*, and create a **Cognitive Services** resource with the following settings:
    - **Name**: *Enter a unique name*.
    - **Subscription**: *Your Azure subscription*.
    - **Location**: *Any available location*.
    - **Pricing tier**: S0
    - **Resource group**: *Create a resource group with a unique name*.
3. Wait for deployment to complete. Then go to your cognitive services resource, and on the **Quick start** page, note the keys and endpoint. You will need these to connect to your cognitive services resource from client applications.
4. Copy the **Key1** for your resource and paste it in the code below, replacing **YOUR_COG_KEY**.
5. Copy the **endpoint** for your resource and and paste it in the code below, replacing **YOUR_COG_ENDPOINT**.
6. Run the code in the cell below by clicking its green <span style="color:green">&#9655</span> button.

> **Note**: If you restart this notebook, you may need to run this cell again to reinitialize these values.

In [ ]:
cog_key = 'YOUR_COG_KEY'
cog_endpoint = 'YOUR_COG_ENDPOINT'

print('Ready to use cognitive services at {} using key {}'.format(cog_endpoint, cog_key))

Now that you've set up the key and endpoint, you can use the custom vision service to analyze an image.

Run the following cell to get a description for an image in the */data/voc/2009_004642.jpg* file.

In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from msrest.authentication import CognitiveServicesCredentials
import matplotlib.pyplot as plt
from PIL import Image
import os
%matplotlib inline

# Get a client for the computer vision service
computervision_client = ComputerVisionClient(cog_endpoint, CognitiveServicesCredentials(cog_key))

# Open an image and display it
image_path = os.path.join('data', 'voc', '2009_004642.jpg')
img = Image.open(image_path)
plt.axis('off')
plt.imshow(img)

# Get a description from the computer vision service
image_stream = open(image_path, "rb")
description_results = computervision_client.describe_image_in_stream(image_stream )
if (len(description_results.captions) == 0):
    print("No description detected.")
else:
    for caption in description_results.captions:
        print("'{}' with confidence {:.2f}%".format(caption.text, caption.confidence * 100))



The description provided seems to be pretty accurate.

The Computer Vision cognitive service offers a lot more functionality than generating image descriptions, including:

- Suggesting "tags" for images, that can be useful if you want to index a lot of images for searching.
- Identifying celebrities or well-known landmarks in images.
- Detecting brand logos in an image.
- Performing optical character recognition (OCR) to read text in an image.
- Detect adult content in an image.

> **Learn More**: To learn more about the Computer Vision cognitive service, see the [Computer Vision documentation](https://docs.microsoft.com/azure/cognitive-services/computer-vision/)

## Using the Custom Vision Cognitive service

The Computer Vision cognitive service provides useful pre-built models for working with images, but you'll often need to train your own model for computer vision. For example, suppose Adventure Works Cycles wants to use the cameras around the city to analyze traffic by identifying images of cars, buses, and cyclists. To do this, you'll need to train a *classification* model that can categorize images into these classes of road user.

### Create a Custom Vision Authoring Resource

The *Custom Vision* service enables you to train image classification and object detection models using your own set of training images, so you can create a model that is tailored for your needs. There are two parts to a custom vision solution - first you must *train* a model using your image data, and then you can *predict* image classes using your trained model. You can use your existing Cognitive Services resource to generate *predictions*, but you must create a separate resource for *training* the model.

1. In another browser tab, open the Azure portal (<a href='https://portal.azure.com' target='_blank'>https://portal.azure.com</a>), signing in with your Microsoft account.
2. Click **+ Create a resource**, and search for *Custom Vision*.
3. In the list of services, click **Custom Vision**.
4. In the **Custom Vision** blade, click **Create**.
5. In the **Create** blade, select only **Training** (you will use your existing cognitive services resource for prediction). Then enter the following details and click **Create**
  * **Name**: A unique name for your service
  * **Subscription**: Your Azure subscription
  * **Resource Group**: The existing resource group you used previously
  * **Training location**: The same location as your cognitive services resource
  * **Training pricing tier**: F0
6. Wait for the service to be created.

### Train an Image Classification Model
Now you can use the *Custom Vision* cognitive service to train your own computer vision model based on existing images.

1. Download and extract the training images from **[here](https://github.com/GraemeMalcolm/ai-fundamentals/raw/master/data/voc/training_images.zip)**.
2. Open the **[Custom Vision portal](https://www.customvision.ai/projects)**. If prompted, sign in using the Microsoft account associated with your Azure subscription.
3. In the Custom Vision portal, create a new project with the following settings:
    - **Name**: Traffic Classification
    - **Description**: Image classification for traffic.
    - **Resource**: *The custom vision resource you created previously in this lab*
    - **Project Types**: Classification
    - **Classification Types**: Multiclass (single tag per image)
    - **Domains**: General
4. Click **\[+\] Add images**, and select all of the files in the **bus** folder you extracted previously. Then upload the image files, specifying the tag *bus*.
5. Repeat the previous step to upload the images in the **car** folder with the tag *car*, and the images in the **cyclist** folder with the tag *cyclist*.
6. Explore the images you have uploaded in the Custom Vision project - there should be 40 images of each class.
7. In the Custom Vision project, click **Train** to train a classification model using the tagged images. Select the **Quick Training** option.
8. Wait for training to complete, and then review the *Precision*, *Recall*, and **AP* performance metrics - these measure the prediction accuracy of the classification model, and should all be high.
9. Click **&#128504; Publish** to publish the trained model with the following settings:
    - **Model name**: traffic
    - **Prediction Resource**: *Your cognitive services resource*.
10. After publishing, click the **&#9881;** icon at the top right to view the project settings, and note the **Project Id**. Copy this value and paste it in the code cell below, replacing **YOUR_PROJECT_ID**; then run the code cell.

In [ ]:
project_id = 'YOUR_PROJECT_ID'
model_name = 'traffic'
print('Ready to predict using model {} in project {}'.format(model_name, project_id))

Now that you have trained and published your custom vision classification model, you can use it from a client application.

Run the following code cell, which uses your model to classifiy a selection of test images.

In [ ]:
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
import matplotlib.pyplot as plt
from PIL import Image
import os
%matplotlib inline

# Get the test images from the data/voc/test folder
test_folder = os.path.join('data', 'voc', 'test')
test_images = os.listdir(test_folder)

# Create an instance of the prediction service
custom_vision_client = CustomVisionPredictionClient(cog_key, endpoint=cog_endpoint)

# Create a figure to display the results
fig = plt.figure(figsize=(16, 8))

# Get the images and show the predicted classes
for idx in range(len(test_images)):
    # Open the image, and use the custom vision model to classify it
    image_contents = open(os.path.join(test_folder, test_images[idx]), "rb")
    classification = custom_vision_client.classify_image(project_id, model_name, image_contents.read())
    # The results include a prediction for each tag, in descending order of probability - get the first one
    prediction = classification.predictions[0].tag_name
    # Display the image with its predicted class
    img = Image.open(os.path.join(test_folder, test_images[idx]))
    a=fig.add_subplot(len(test_images)/3, 3,idx+1)
    a.axis('off')
    imgplot = plt.imshow(img)
    a.set_title(prediction)
plt.show()

The Custom Vision cognitive service can classify new images based on the tags you specified in the training data. You can also use the Custom Vision service to create *object detection* models, which not only classify objects in images, but also identify *bounding boxes* that show the location of the object in the image.

> **Learn More**: To learn more about the Custom Vision cognitive service, view the [Custom Vision documentation](https://docs.microsoft.com/azure/cognitive-services/custom-vision-service/home).

## Using the Face Cognitive service

You may have noticed that the Custom Vision model you trained actually identifies *cycles* rather than *cyclists*. It might be useful to extend the traffic analysis application to analyze images that are classified as *cyclist* to determine if they contain any human faces; and if so count the number of faces detected and highlight them in the image.

To accomplish this, you'll use a third cognitive service that provides face detection and facial recognition capabilies.

The code below performs the same image classification as previously, but now when a *cyclist* image is found, the code uses the **Face** cognitive service to detect faces in the image.

Run the code cell below to see the results of this enhancement to the application.

In [ ]:
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.face import FaceClient
from msrest.authentication import CognitiveServicesCredentials
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import os
%matplotlib inline

# Get the test images from the data/voc/test folder
test_folder = os.path.join('data', 'voc', 'test')
test_images = os.listdir(test_folder)

# Create a prediction client
custom_vision_client = CustomVisionPredictionClient(cog_key, endpoint=cog_endpoint)

# Create a face detection client.
face_client = FaceClient(cog_endpoint, CognitiveServicesCredentials(cog_key))

# Create a figure to display the results
fig = plt.figure(figsize=(16, 16))

# Get the images and show the predicted classes
for idx in range(len(test_images)):
    # Open the image, and use the custom vision model to classify it
    image_contents = open(os.path.join(test_folder, test_images[idx]), "rb")
    classification = custom_vision_client.classify_image(project_id, model_name, image_contents.read())
    # The results include a prediction for each tag, in descending order of probability - get the first one
    prediction = classification.predictions[0].tag_name
    # Open the image so we can add it to the figure
    img = Image.open(os.path.join(test_folder, test_images[idx]))

    # If the image is a cyclist, detect faces
    if prediction == 'cyclist':
        image_stream = open(os.path.join(test_folder, test_images[idx]), "rb")
        detected_faces = face_client.face.detect_with_stream(image=image_stream)
        if detected_faces:
            # If there are faces, how many?
            num_faces = len(detected_faces)
            prediction = prediction + ' (' + str(num_faces) + ' faces detected)'
            # Draw a rectangle around each detected face
            for face in detected_faces:
                r = face.face_rectangle
                bounding_box = ((r.left, r.top), (r.left + r.width, r.top + r.height))
                draw = ImageDraw.Draw(img)
                draw.rectangle(bounding_box, outline='magenta', width=5)

    # Display the image with its predicted class and detected faces
    a=fig.add_subplot(len(test_images)/3, 3,idx+1)
    a.axis('off')
    imgplot = plt.imshow(img)
    a.set_title(prediction)
plt.show()

The Face cognitive service can do much more than simply detect faces. It can also analyze facial features and expressions to suggest gender, age, and emotional state; and it can compare faces for similarity and be trained to recognize individual faces.

> **Learn More**: To learn more about the Face cognitive service, see the [Face documentation](https://docs.microsoft.com/azure/cognitive-services/face/)

## Use Optical Character Recognition to Read text in Images

So far you've seen how you can use congitive services in Azure to analyze and classifiy images, and identify faces in photographs. Another common computer vision challenge is to detect and interpret text in an image. This kind of processing is often referred to as *optical character recognition* (OCR).

### Read Text in an Image
Let's start with a simple example. The **Computer Vision** cognitive service has some built-in OCR capabilities that enable you to detect the location of text in an image. Let's put it to the test to see if the Adventure Works street cameras can read text on road signs.


In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from msrest.authentication import CognitiveServicesCredentials
import matplotlib.pyplot as plt
from PIL import Image
import os
%matplotlib inline

# Get a client for the computer vision service
computervision_client = ComputerVisionClient(cog_endpoint, CognitiveServicesCredentials(cog_key))

# Create a figure to display the results
image_path = os.path.join('data', 'sign.jpg')
img = Image.open(image_path)
plt.axis('off')
plt.imshow(img)
image_stream = open(image_path, "rb")
ocr_results = computervision_client.recognize_printed_text_in_stream(image_stream)
for region in ocr_results.regions:
    for line in region.lines:
        for word in line.words:
            print(word.text)


### Extract Information from Forms

A more advanced OCR scenario is the extraction of information from forms or invoices. The **Form Recognizer** service is specifically designed for this kind of AI problem.

In this example, you'll use the Form Recognizer's built-in model for analyzing receipts.

Start by creating a Form Recognizer resource in your Azure subscription:

1. In another browser tab, open the Azure portal (<a href='https://portal.azure.com' target='_blank'>https://portal.azure.com</a>), signing in with your Microsoft account.
2. Select **+ Create a resource**, and search for *Form Recognizer*.
3. In the list of services, select **Form Recognizer**.
4. In the **Form Recognizer** blade, select **Create**.
5. In the **Create** blade, enter the following details and select **Create**
  * **Name**: A unique name for your service
  * **Subscription**: Your Azure subscription
  * **Location**: Any available location
  * **Pricing tier**: F0
  * **Resource Group**: The existing resource group you used previously
  * **I confirm I have read and understood the notice below**: Selected.
6. Wait for the service to be created.
7. View your newly created Form Recognizer service in the Azure portal and on the **Quick Start** page, copy the **Key1** and **Endpoint** values and paste them in the code cell below.

In [ ]:
form_key = '827d9c9bbf3a48cc99f66301ddab5d1c'
form_endpoint = 'https://gmalc-form.cognitiveservices.azure.com/'
print('Ready to use form recognizer at {} using key {}'.format(form_endpoint, form_key))

Now you're ready to use Form Recognizer to analyze a receipt.

In [ ]:
import json
import time
from requests import get, post
import matplotlib.pyplot as plt
from PIL import Image
import os
%matplotlib inline


# Create a figure to display the results
fig = plt.figure(figsize=(8, 8))
image_path = os.path.join('data', 'receipt.jpg')
img = Image.open(image_path)
plt.axis('off')
plt.imshow(img)

post_url = form_endpoint + "/formrecognizer/v2.0-preview/prebuilt/receipt/analyze"

headers = {
    'Content-Type': 'image/jpeg',
    'Ocp-Apim-Subscription-Key': form_key,
}

params = {
    "includeTextDetails": True
}

with open(image_path, "rb") as f:
    data_bytes = f.read()

try:
    resp = post(url = post_url, data = data_bytes, headers = headers, params = params)
    if resp.status_code != 202:
        print("POST analyze failed:\n%s" % resp.text)
        quit()
    print("POST request succeeded. Awaiting analysis results...")
    get_url = resp.headers["operation-location"]
except Exception as e:
    print("POST analyze failed:\n%s" % str(e))
    quit()

n_tries = 10
n_try = 0
wait_sec = 6
complete = False
while n_try < n_tries and complete != True:
    try:
        resp = get(url = get_url, headers = {"Ocp-Apim-Subscription-Key": form_key})
        resp_json = json.loads(resp.text)
        if resp.status_code != 200:
            print("GET Layout results failed:\n%s" % resp_json)
            complete = True
        status = resp_json["status"]
        if status == "succeeded":
            fields = resp_json["analyzeResult"]["documentResults"][0]["fields"]
            for field in fields:
                fieldType = fields[field]["type"]
                if fieldType != 'array':
                    fieldTypeString = fieldType[0].capitalize() + fieldType[1:]
                    valueField = 'value' + fieldTypeString
                    print(field, fields[field][valueField])
            complete = True
        if status == "failed":
            print("Analysis failed:\n%s" % resp_json)
            complete = True
        # Analysis still running. Wait and retry.
        time.sleep(wait_sec)
        n_try += 1     
    except Exception as e:
        msg = "GET analyze results failed:\n%s" % str(e)
        print(msg)
        complete = True